<h1>Ozempic Side Effects NLP<h1/>

<h2>Libraries<h2/>

In [57]:
import pandas as pd
import nltk
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from wordcloud import WordCloud

<h2>hello<h2/>

In [39]:
# PRAW Setup
reddit = praw.Reddit(
    client_id='FsHBk7J5OcNXIh2OmzXWOg',
    client_secret='oFwrVTKJgbRcjslvDCp0iM6d9RO7Xw',
    user_agent='MB WebScrapping by u/ActualConfusion3366'
)

count = 0
comments = []
for post in reddit.subreddit('all').search("ozempic", sort="new", limit=None):
    post.comments.replace_more(limit=0)
    """print(f"Title: {post.title}")
    print(f"Score: {post.score}")
    print(f"URL: {post.url}")
    print("-" * 50)"""
    count += 1
    for comment in post.comments[:5]:
        comments.append(comment.body)


df = pd.DataFrame(comments, columns=["comment"])   
print(df.head())
print(count)


                                             comment
0  If you’ve been off of any GLP-1 type peptide f...
1  Is he the jerk it sounds like he is?  He must ...
2  Honestly that’s heartbreaking her mom does tha...
3  No. As dogs do not prepare their own food, it'...
4  No.\n\nDo you need help building a diet plan f...
246


<h2>hello<h2/>

In [54]:
# Download NLTK stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Step 1: Text Preprocessing
def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Remove punctuation
    text = ''.join([char for char in text if char not in string.punctuation])
    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

df = df[~df['comment'].isin(["[deleted]", "[removed]"])]
df['cleaned_post'] = df['comment'].apply(preprocess_text)
df

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mab23\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,comment,cleaned_post
0,If you’ve been off of any GLP-1 type peptide f...,you’ve glp1 type peptide 3 months act like you...
1,Is he the jerk it sounds like he is? He must ...,jerk sounds like must feel like loser failure ...
2,Honestly that’s heartbreaking her mom does tha...,honestly that’s heartbreaking mom
3,"No. As dogs do not prepare their own food, it'...",dogs prepare food easy put weight loss diet se...
4,No.\n\nDo you need help building a diet plan f...,need help building diet plan dog hypothetical
...,...,...
668,I need this shirt I hope they release it again!,need shirt hope release
669,Any notable departures in their patent departm...,notable departures patent department last year...
671,"Fortune falsely cites ""a recent report in *Sci...",fortune falsely cites recent report science ac...
672,![gif](giphy|GDnomdqpSHlIs),gifgiphygdnomdqpshlis


<h2>hello<h2/>

In [58]:
# Step 2: TF-IDF Vectorization
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['cleaned_post'])
X

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 15511 stored elements and shape (668, 4896)>

<h2>hello<h2/>

In [64]:
# Step 3: KMeans Clustering
kmeans = KMeans(n_clusters=5, random_state=42)
df['cluster'] = kmeans.fit_predict(X)
df

,comment,cleaned_post,cluster,sentiment,predicted_sentiment
0,If you’ve been off of any GLP-1 type peptide f...,you’ve glp1 type peptide 3 months act like you...,1,1,1
1,Is he the jerk it sounds like he is? He must ...,jerk sounds like must feel like loser failure ...,0,0,1
2,Honestly that’s heartbreaking her mom does tha...,honestly that’s heartbreaking mom,0,0,0
3,"No. As dogs do not prepare their own food, it'...",dogs prepare food easy put weight loss diet se...,3,1,1
4,No.\n\nDo you need help building a diet plan f...,need help building diet plan dog hypothetical,1,1,1
...,...,...,...,...,...
668,I need this shirt I hope they release it again!,need shirt hope release,1,0,1
669,Any notable departures in their patent departm...,notable departures patent department last year...,1,1,1
671,"Fortune falsely cites ""a recent report in *Sci...",fortune falsely cites recent report science ac...,0,0,0
672,![gif](giphy|GDnomdqpSHlIs),gifgiphygdnomdqpshlis,0,0,0


<h2>hello<h2/>

In [66]:
# Step 4: Sentiment Classification (Optional)
# Dummy sentiment labels (replace with actual labels if available)
df['sentiment'] = df['cluster']  # 1: Positive, 0: Negative

# Train a Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X, df['sentiment'])

# Predict sentiment
df['predicted_sentiment'] = clf.predict(X)

# Classification report
report = classification_report(df['sentiment'], df['predicted_sentiment'], target_names=['Negative', 'Positive'])
print(report)
df

ValueError: Number of classes, 5, does not match size of target_names, 2. Try specifying the labels parameter

In [67]:
# Step 4: Sentiment Classification (Optional)
# Dummy sentiment labels (replace with actual labels if available)
df['sentiment'] = df['cluster']  # 1: Positive, 0: Negative

# Train a Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X, df['sentiment'])

# Predict sentiment
df['predicted_sentiment'] = clf.predict(X)

# Classification report
report = classification_report(df['sentiment'], df['predicted_sentiment'], target_names=['Negative', 'Positive', 'Neutral', 'Mixed', 'Other'])
print(report)
df

              precision    recall  f1-score   support

    Negative       0.99      0.79      0.88       171
    Positive       0.52      1.00      0.69       268
     Neutral       1.00      0.11      0.20        81
       Mixed       1.00      0.03      0.06        60
       Other       1.00      0.10      0.19        88

    accuracy                           0.63       668
   macro avg       0.90      0.41      0.40       668
weighted avg       0.81      0.63      0.56       668



,comment,cleaned_post,cluster,sentiment,predicted_sentiment
0,If you’ve been off of any GLP-1 type peptide f...,you’ve glp1 type peptide 3 months act like you...,1,1,1
1,Is he the jerk it sounds like he is? He must ...,jerk sounds like must feel like loser failure ...,0,0,1
2,Honestly that’s heartbreaking her mom does tha...,honestly that’s heartbreaking mom,0,0,0
3,"No. As dogs do not prepare their own food, it'...",dogs prepare food easy put weight loss diet se...,3,3,1
4,No.\n\nDo you need help building a diet plan f...,need help building diet plan dog hypothetical,1,1,1
...,...,...,...,...,...
668,I need this shirt I hope they release it again!,need shirt hope release,1,1,1
669,Any notable departures in their patent departm...,notable departures patent department last year...,1,1,1
671,"Fortune falsely cites ""a recent report in *Sci...",fortune falsely cites recent report science ac...,0,0,0
672,![gif](giphy|GDnomdqpSHlIs),gifgiphygdnomdqpshlis,0,0,0
